In [18]:
import numpy as np
import pandas as pd
import spacy
from spacy.matcher import Matcher
from spacy.tokens import Span
from spacy import displacy
from collections import defaultdict
nlp = spacy.load('en_core_web_md')
from string import punctuation

In [31]:
df = pd.read_json('../gg2013.json')
df = df['text']
df = df[~df.str.contains('think|should|maybe', case = False)]
df = df.str.replace('#GoldenGlobes|#GoldenGlobe|Golden Globe|Golden Globes', "", case = False, regex = False)
best_df = df[df.str.contains('Best', case = True)]
# best_df = df[df.str.contains('win', case = True)]
if best_df.shape[0] > 4000:
    best_df = best_df.sample(n=4000)
best_df.shape[0]

4000

In [32]:
pattern = [{"ORTH":'Best'}, {"DEP": 'compound', 'OP':'+'}, {"POS":'NOUN', 'OP':'*'}, {'IS_PUNCT': True, 'OP':'?'}, {"POS": 'PROPN', 'OP':'*'}]

In [33]:
matcher = Matcher(nlp.vocab)
matcher.add('Winner', None, pattern)

In [34]:
doc = nlp('Sam wong wins Best Motion Picture – Foreign Language')

In [35]:
matches = matcher(doc)

In [36]:
matches[-1][2]

9

In [37]:
for entity in doc:
    print(entity.text, entity.pos_,entity.dep_)

Sam PROPN compound
wong PROPN nsubj
wins VERB ROOT
Best PROPN compound
Motion PROPN compound
Picture NOUN dobj
– PUNCT punct
Foreign PROPN compound
Language PROPN appos


In [38]:
for match_id, start, end in matches:
    string_id = nlp.vocab.strings[match_id]
    span = doc[start:end]
    print(match_id, string_id, start, end, span.text)

7971871675768048331 Winner 3 5 Best Motion
7971871675768048331 Winner 3 6 Best Motion Picture
7971871675768048331 Winner 3 7 Best Motion Picture –
7971871675768048331 Winner 3 8 Best Motion Picture – Foreign
7971871675768048331 Winner 3 9 Best Motion Picture – Foreign Language


In [46]:
awards = {}
awards = defaultdict(lambda: 1, awards)
for i, text in best_df.iteritems():
    doc = nlp(text)
    matches = matcher(doc)
    if len(matches) != 0:
        span = doc[matches[-1][1]:matches[-1][2]]
        awards[str(span).strip(punctuation).strip()] = awards[str(span).strip(punctuation).strip()] +1

In [47]:
awards = sorted(awards.items(), key=lambda item: item[1], reverse = True)
print(awards[0:25])

[('Best Supporting Actress', 322), ('Best Original Song', 205), ('Best Motion Picture', 147), ('Best Supporting Actor', 120), ('Best Motion Picture - Comedy', 67), ('Best Screenplay - Quentin Tarantino', 49), ('Best Television Series', 49), ('Best Animated Feature Film - Brave', 38), ('Best Original Score - Mychael Danna', 35), ('Best TV Series', 34), ('Best Comedy Series', 29), ('Best Actress Award', 27), ('Best Television Series Actor - Drama', 27), ('Best Motion Picture, Musical', 26), ('Best Golden Globes', 26), ('Best Original Score', 25), ('Best Original Song: Adele', 25), ('Best motion picture drama', 24), ('Best Foreign Film - Amour', 23), ('Best Motion Picture - Drama', 22), ('Best Television Series - Comedy', 22), ('Best Motion Picture Drama', 20), ('Best Foreign Film', 18), ('Best TV Series Actress - Comedy', 18), ('Best Director award', 18)]
